<h1> Getting Started on a Cipher Diagnostic Tool</h1>

<p>One of the most challenging aspects of cryptanalysis (at least with classical ciphers) is the identification of the type of cipher. Is it monoalphabetic, polygraphic, homophonic, or something else entirely?

Once the correct type of cipher has been identified, the real work can start in earnest. This is easier said than done. 

Let's see if we can work toward a cipher classifier. This is going to take a lot of work, but with persistence and patience, I think we can produce something useful... but we need some data. In particular, we need to identify features of ciphers that can be used be the classifier in learning the structure of different ciphers.

There are some clear choices. The ciphertext itself, and Friedman's Index of Coincidence (IoC) for a ciphertext are the most obvious starting places. Other important measures might include the Shannon entropy (though there may be some dependence between this and the IoC, so we will need to be careful), and the ngram frequency distributions. Polygraphic substitutions and homophonic substitutions can have a far fewer or greater number of characters that the standard English alphabet. It may be tempting to include known plaintext as well, but most cipher diagnosis  Let's start making a list.

<ul>
    <li>The Ciphertext Itself</li>
    <li>Index of Coincidence</li>
    <li>Shannon Entropy of the Ciphertext</li>
    <li>N-Gram Frequency Distributions</li>
    <li>Factors of Numbers Close to the Ciphertext Length</li>
    <li>Other Things I Have Not Thought Of Yet!</li>

</ul>

This, admittedly, is not a large number of features to consider. We will keep our minds open.

However, we are still getting ahead of ourselves! We don't even have any ciphers to work with! We also need to identify the kinds of assumptions that will underly our model, as that will impact the ways in which we construct our dataset.
</p>

<h2>Assumptions</h2>
Given a random ciphertext, how likely is it that is was encrypted with a Affine Cipher? Vigenere? Hill? 

Well, as I know of no "Complete Database of Classical Ciphers," and since we want the classifier to base its conclusion on the characteristics of a given ciphertext, we will create this dataset in such a way that ciphers are approximately uniformly distributed. That is, a given ciphertext in the dataset will have an equal chance of having been enciphered with any of the encryption algorithms implemented.

Furthermore, the likelihood of a ciphertext being encrypted with a given cipher should be independent of the length of the ciphertext. So we should be certain that ciphertext lengths be distributed among individual ciphers in approximately the same way as well. 

<ol>
    <li>The distribution of ciphers should be approximately unifor .</li>
    <li>The distribution of lengths of a ciphertexts should be approximately uniform.</li>
</ol>

This looks like a good starting place, and we can return to these assumptions later if we feel so inclined. Let's make a dataset!

<h2>Data</h2>

For the time being, we are going to focus on ciphertext that has been encrypted from English plaintext. As such, Project Gutenburg seems like a good resource. Given their enormous influence in other texts, why don't we start with the top 100 downloaded ebooks of the last 30 days? 

It will probably be usefule to create regex object to identify the appropriate links on the webpage. A little poking around the source on https://www.gutenberg.org/browse/scores/top should help us identify the correct format.

There appear to be a few ways the download urls appear. Here are the ones I have found so far (note I have formatted them according to
the structure of f-strings):
<ul>
    <li>"https://www.gutenberg.org/files/{ebook_id}/{ebook_id}.txt"</li>
    <li>"https://www.gutenberg.org/cache/epub/{ebook_id}/pg{ebook_id}.txt"</li>
    <li>"https://www.gutenberg.org/files/{ebook_id}/{ebook_id}-0.txt"</li>
    <li>"https://www.gutenberg.org/cache/epub/{ebook_id}/pg{ebook_id}-0.txt"</li>

</ul>

Okay! Now let's create some functions to do the heavy lifting for us! First we will write a function to connect to the appropriate website. Then a second function can be used to parse the html and find matches for the html elements that match the regular expression above, and return a list with all of the book names and ID numbers.

After this, we will write a third function will construct urls for downloading each of the books with IDs retreived and a fourth function will attempt to download each of the books, trying a variety of file extensions and locations if the download fails.

In [65]:
# A function to connect to the top 100 ebooks page on Project Gutenberg and return the html
def get_top_100_ebooks_html():
    """
    Attempts to connect to the top 100 ebooks page on Project Gutenberg and returns the html. Returns None if an error.
    """

    # Import the requests module
    import requests

    # Try to connect to the top 100 ebooks page on Project Gutenberg and return the html
    try:
        return requests.get("https://www.gutenberg.org/browse/scores/top").text
    # If an error occurs, return None
    except requests.exceptions.RequestException as e:
        print(e)
        return None

# A function to parse the html and return a list of tuples containing the ebook ID and title
def get_top_100_ebooks(force_duplicate_removal: bool = True):
    """
    Parses the html of the top 100 ebooks page on Project Gutenberg using regular expressions and returns a list of
    tuples containing the ebook ID and title. If force_duplicate_removal is True, duplicates with the same name will be
    removed. Otherwise, the user will be asked which duplicate to remove (if any exist). 

    Args:
        force_duplicate_removal (bool): Whether to force the removal of duplicates with the same name. Defaults to True.

    Returns:
        list[tuple[str, str]]: A list of tuples containing the ebook ID and title.

    Raises:
        ValueError: If the input parameters are not in the expected format.

    """

    # The regular expression to match
    html_element_regex = re.compile(r'<a href="/ebooks/(\d+)">(.+)</a>')

    # Get the html of the top 100 ebooks page on Project Gutenberg
    html = get_top_100_ebooks_html()

    # Find all matches of the html_element_regex in the html
    matches = html_element_regex.findall(html)

    # Simplify the names of the ebooks just the first 6 words of the name (at a maximum)
    simplified_matches = [(ebook_id, " ".join(name.split()[:6])) for ebook_id, name in matches]

    # Check for matches with different IDs but the same name
    for i in range(len(simplified_matches)):
        for j in range(i + 1, len(simplified_matches)):
            if simplified_matches[i][1] == simplified_matches[j][1]:
                print(f"Duplicates found: {simplified_matches[i][0]} and {simplified_matches[j][0]} ")
                # If force_duplicate_removal is True, remove the duplicate with the higher ID
                if force_duplicate_removal:
                    simplified_matches.pop(j)

    # Return the simplified matches
    return simplified_matches

# A function to download an ebook given its ID
def download_ebook(ebook_id: str):
    """
    Downloads an ebook given its ID, and saves it to the text_files directory.

    Args:
        ebook_id (str): The ID of the ebook to download.
    """

    # Import the requests module
    import requests

    # Possible urls
    urls = [f"https://www.gutenberg.org/files/{ebook_id}/{ebook_id}.txt", f"https://www.gutenberg.org/cache/epub/{ebook_id}/pg{ebook_id}.txt",
            f"https://www.gutenberg.org/files/{ebook_id}/{ebook_id}-0.txt", f"https://www.gutenberg.org/cache/epub/{ebook_id}/pg{ebook_id}-0.txt"]

    # Try each url until one works, and save the text to a .txt file in the text_files directory
    for url in urls:
        try:
            return requests.get(url).text
        except:
            pass

    # If none of the urls worked, return None
    return None

def ebook_downloaded(ebook_title: str) -> bool:
    """
    Returns whether an ebook has been downloaded by checking for existing variations of the name already within the text_files directory.
    """

    # Import the os and re modules
    import os
    import re

    ebook_title_regex = re.compile(rf"{ebook_title}(\s\(\d+\))?\.txt")

    # Return whether any files in the text_files directory match the ebook_title_regex
    return any(ebook_title_regex.match(filename) for filename in os.listdir("text_files"))


# A function to download the top 100 ebooks
def download_top_100_ebooks():
    """
    Downloads the top 100 ebooks.
    """

    # Get the top 100 ebooks
    top_100_ebooks = get_top_100_ebooks()

    # Download each ebook
    for ebook_id, ebook_title in top_100_ebooks:
        print(f"Downloading {ebook_title}...")
        ebook_text = download_ebook(ebook_id)
        if ebook_text is not None and not ebook_downloaded(ebook_title):
            with open(f"{ebook_title}.txt", "w") as f:
                f.write(ebook_text)


In [66]:
# Time to download the top 100 ebooks!
download_top_100_ebooks()

<h3>Success!... And More Issues to Address</h3>
It looks as if things worked more or less according to plan. We have some things to deal with, like headers and footers to each text file, and there are some books which are not in English (we can try an incorporate non-English text later on).

The style of writing (vocabularly, grammar, etc.) in these books is remarkably different and they come in a variety of lengths. For any cipher that is neither polyalphabetic nor polygraphic, the usage of different symbols, and therefore the vernacular of the period in which a text was written, will likely alter predictions due to differences in n-gram frequencies, particularly with $n > 1$. As such, we need to be sure that each book has an equal chance of being enciphered with a given cipher. In addition, since there is no reason someone cannot encipher sentence fragments, paraphrases, shorthand, or otherwise truncated text, we should try to ensure some of the plaintext possesses these characteristics.

Let's create a function to remove the boilerplate text from the top and bottom of each ebook text file.

In [55]:
# A function to remove the boilerplate text from the Project Gutenberg books.
import os
import re

def clean_text_files():
    """
    Removes the boilerplate text from the Project Gutenberg books using regular expression matching.
    
    """

    # Get a list of all the files in the text_files director with a .txt file extension.
    files = os.listdir("text_files")
    files = [file for file in files if file.endswith(".txt")]
        
    # A regular expression to match the boilerplate text at the beginning of Project Gutenberg books.
    boilerplate_regex = re.compile(r'\*\*\* START OF THE PROJECT GUTENBERG EBOOK .+ \*\*\*')

    # A regular expression to match the boilerplate text at the end of Project Gutenberg books.
    end_boilerplate_regex = re.compile(r'\*\*\* END OF THIS PROJECT GUTENBERG EBOOK .+ \*\*\*')

    # Remove the boilerplate text from the Project Gutenberg books.
    for file in files:
        # Open the file.
        with open("text_files/" + file, "r") as f:
            text = f.read()

        # Remove the boilerplate text at the beginning of the file.
        text = re.sub(boilerplate_regex, '', text)

        # Remove the boilerplate text at the end of the file.
        text = re.sub(end_boilerplate_regex, '', text)

        # Save the file.
        with open("text_files/" + file, "w") as f:
            f.write(text)

In [56]:
# A function that finds and removes any html files from the text_files directory.
import os
import re

def find_and_remove_html_files():
    # A regular expression to locate html in a file.
    html_regex = re.compile(r'<.+>')

    # Get a list of all the files in the text_files director with a .txt file extension.
    files = os.listdir("text_files")
    files = [file for file in files if file.endswith(".txt")]

    # Find the files that contain html.
    html_files = []
    for file in files:
        # Open the file.
        with open("text_files/" + file, "r") as f:
            text = f.read()

        # Check whether the file contains html.
        if html_regex.search(text):
            html_files.append(file)

    # Remove the html files.
    for file in html_files:
        os.remove("text_files/" + file)
        


In [ ]:
# A function to recognize English text.
import nltk
from nltk.corpus import words
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
import string
import random
import math
import re

def is_english(text: str, threshold: float = 0.9) -> bool:
    """
    Returns True if the text is English with certainty greater than threshold and False otherwise.

    Args:
        text (str): The text to be analyzed.
        threshold (float): The certainty threshold above which the text is considered English.

    Returns:
        bool: True if the text is English with certainty greater than threshold and False otherwise.
    
    Raises:
        ValueError: If threshold is not a float between 0 and 1.
    
    """

    # Check that threshold is a float between 0 and 1.
    if not isinstance(threshold, float) or threshold < 0 or threshold > 1:
        raise ValueError("threshold must be a float between 0 and 1.")
    
    

To this end, let's create a function to randomly select a piece of text (e.g., paragraphs, sentences, fragments) from one of these books, which we will also select randomly. Let's also try to ensure it doesn't cut words into pieces.

In [47]:
# A function to randomly select a book from the text_files directory, and then randomly select a bit of text from that book,
# ensuring that the text does not cut words into pieces.
import random
def generate_random_text_samples(file_path: str, num_samples=1) -> list:
    """
    Generates random text samples from the books in the text_files directory. The samples are randomly selected from the books. 
    The samples are guaranteed to be complete words, and the number of samples is determined by the num_samples parameter.   

    Args:
        file_path (str): The path to the directory containing the text files.
        num_samples (int): The number of samples to generate. Defaults to 1.

    Returns:
        list: A list of strings containing the text samples.

    Raises:
        ValueError: If num_samples is not an integer greater than 0.

    """

    # Check that num_samples is an integer greater than 0.
    if num_samples < 1 or not isinstance(num_samples, int):
        raise ValueError("num_samples must be an integer greater than 0.")
    
    # Get a list of all the text files in the directory.
    text_files = os.listdir(file_path)

    # Select a random text file.
    text_file = random.choice(text_files)

    # Read the text file.
    with open(file_path + "/" + text_file, "r") as f:
        text = f.read()

    # Split the text into words.
    words = text.split()

    # Remove punctuation from the words.
    words = [re.sub(r'[^\w\s]', '', word) for word in words]

    # Remove empty strings from the words.
    words = [word for word in words if word != ""]

    # Remove numbers from the words.
    words = [word for word in words if not word.isdigit()]

    # Loop until we have the desired number of samples.
    samples = []
    while len(samples) < num_samples:
        # Select a random word.
        word = random.choice(words)

        # Get the index of the word.
        index = words.index(word)

        # Get the number of words in the text.
        num_words = len(words)

        # Get the number of words in the sample.
        num_sample_words = random.randint(1, len(words)+1)

        # Get the start and end indices of the sample.
        start_index = index - num_sample_words // 2
        end_index = index + num_sample_words // 2
    
        # If the sample would start before the beginning of the text, set the start index to 0.
        if start_index < 0:
            start_index = 0

        # If the sample would end after the end of the text, set the end index to the end of the text.
        if end_index > num_words:
            end_index = num_words

        # Get the sample.
        sample = words[start_index:end_index]

        # Join the words in the sample together.
        sample = " ".join(sample)

        # Add the sample to the list of samples.
        samples.append(sample)

    # Return the samples.
    return samples

Let's test this out by trying to produce 1000 text samples from our books.

In [48]:
# Generate 10 random text samples using our new function.
samples = generate_random_text_samples("text_files", 10)

In [50]:
samples

['library of free eBooks meta propertyfbadmins content615269807 meta propertyfbapp_id content115319388529183 meta propertyogsite_name contentProject Gutenberg meta propertyogimage contenthttpswwwgutenbergorggutenbergpglogo144x144png head body div classcontainer start body nav div idmain_logo a idmain_logo href classnohover img srcgutenbergpglogo129x80png altProject Gutenberg draggablefalse a div div',
 'DOCTYPE html html classclientnojs langen dirltr head meta charsetUTF8 title404 Project Gutenbergtitle link relstylesheet hrefgutenbergstylecssv11 link relstylesheet hrefgutenbergcollapsiblecss11 link relstylesheet hrefgutenbergnew_navcssv1321231 link relstylesheet hrefgutenbergpgdesktoponecss meta nameviewport contentwidthdevicewidth initialscale1 meta namekeywords contentbooks ebooks free kindle android iphone ipad meta namegooglesiteverification contentwucOEvSnj5kP3Ts_36OfP64laakK1mVTgptrGC9io meta namealexaVerifyID content4WNaCljsEA82vP_ih2H_UqXZvM link relcopyright hrefhttpswwwgnuor

In [2]:
def generate_ciphertext_plaintext_pairs(cipher: Cipher, input_file: str, output_file: str) -> None:
    """
    Reads a text file, and then generates a file containing ciphertext and plaintext pairs of random length and randomly
    selected starting positions. The ciphertext is constructed using a given cipher object.

    Args:
        cipher (Cipher): The cipher object to use for encryption.
        input_file (str): The name of the file to read.
        output_file (str): The name of the file to write.
    """

    # Read the input file.
    with open(input_file, "r") as f:
        text = f.read()

    # Generate ciphertext and plaintext pairs.
    ciphertext_plaintext_pairs = []
    for i in range(1000):
        # Generate a random length for the ciphertext and plaintext.
        length = random.randint(10, 1000)

        # Generate a random starting position for the ciphertext and plaintext.
        start = random.randint(0, len(text) - length)
        
        # Generate the ciphertext and plaintext. 
        plaintext = text[start:start+length]
        ciphertext = cipher.encrypt(plaintext)

        # Add the ciphertext and plaintext to the list of pairs.
        ciphertext_plaintext_pairs.append((ciphertext, plaintext))

    # Write the ciphertext and plaintext pairs to the output file.
    with open(output_file, "w") as f:
        for pair in ciphertext_plaintext_pairs:
            f.write(pair[0] + "\n")
            f.write(pair[1] + "\n")